In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !unzip "/content/drive/MyDrive/DataScience/Pyimagesearch_DeepLearningBook/Chapter7/archive.zip" -d "/content/drive/MyDrive/DataScience/Pyimagesearch_DeepLearningBook/Chapter7/"

In [3]:
import os

all_images = []

for path, subdirs, files in os.walk("/content/drive/MyDrive/DataScience/Pyimagesearch_DeepLearningBook/Chapter7/animals"):
    for name in files:
        all_images.append(os.path.join(path, name))

len(all_images)

3000

In [4]:
import cv2

class SimplePreprocessor:
  def __init__(self, width, height, inter=cv2.INTER_AREA):
    self.width = width
    self.height = height
    self.inter = inter

  def preprocess(self, image):
    return cv2.resize(image, (self.width, self.height), interpolation = self.inter)

In [5]:
import numpy as np
class SimpleDatasetLoader:
  def __init__(self, preprocessors=None):
    self.preprocessors = preprocessors

    if self.preprocessors is None:
      self.preprocessors = []

  def load(self, imagePaths, verbose=-1):
    data = []
    labels = []

    for (i, imagePath) in enumerate(imagePaths):
      image = cv2.imread(imagePath)
      label = imagePath.split(os.path.sep)[-2]

      if self.preprocessors is not None:
        for p in self.preprocessors:
          image = p.preprocess(image)

      data.append(image)
      labels.append(label)

      if verbose > 0 and i > 0 and (i+1) % verbose == 0:
        print("[INFO] processed {}/{}".format(i+1, len(imagePaths)))

    return (np.array(data), np.array(labels))


In [6]:
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [7]:
print("[INFO] loading images..")

sp = SimplePreprocessor(32, 32)

sdl = SimpleDatasetLoader(preprocessors=[sp])

(data, labels) = sdl.load(all_images, verbose=500)

print(data.shape)

data = data.reshape((data.shape[0], 3072))

print("[INFO] feature matrix: {:.1f}MB".format(data.nbytes / (1024 * 1000.0)))

[INFO] loading images..
[INFO] processed 500/3000
[INFO] processed 1000/3000
[INFO] processed 1500/3000
[INFO] processed 2000/3000
[INFO] processed 2500/3000
[INFO] processed 3000/3000
(3000, 32, 32, 3)
[INFO] feature matrix: 9.0MB


In [8]:
le = LabelEncoder()
labels = le.fit_transform(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [10]:
for r in (None, 'l1', 'l2'):
  print("[INFO] training model with '{}' penalty".format(r))
  model = SGDClassifier(loss='log', penalty=r, max_iter=10, learning_rate='constant', eta0=0.01, random_state=42)

  model.fit(trainX, trainY)

  acc = model.score(testX, testY)
  print("[INFO] '{}' penalty accuracy: {:0.2f}%".format(r, acc * 100))

[INFO] training model with 'None' penalty


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


[INFO] 'None' penalty accuracy: 43.20%
[INFO] training model with 'l1' penalty


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


[INFO] 'l1' penalty accuracy: 42.67%
[INFO] training model with 'l2' penalty
[INFO] 'l2' penalty accuracy: 53.20%


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


In [11]:
print(classification_report(testY, model.predict(testX), target_names= le.classes_))

              precision    recall  f1-score   support

        cats       0.50      0.57      0.53       262
        dogs       0.45      0.24      0.31       249
       panda       0.59      0.79      0.68       239

    accuracy                           0.53       750
   macro avg       0.52      0.53      0.51       750
weighted avg       0.51      0.53      0.51       750

